# Chapter 19 - Facebook's Prophet

## Listing 19-1. Preparing the dependent variable

In [ ]:
import pandas as pd
y = pd.read_csv('air_visit_data.csv.zip')
y = y.pivot(index='visit_date', columns='air_store_id')['visitors']
y = y.fillna(0)
y = pd.DataFrame(y.sum(axis=1))


## Listing 19-2. Preparing the modeling data frame

In [ ]:
y = y.reset_index(drop=False)
y.columns = ['ds', 'y']


## Listing 19-3. Creating a train-test split

In [ ]:
train = y.iloc[:450,:]
test = y.iloc[450:,:]


## Listing 19-4. Creating a basic Prophet model

In [ ]:
from fbprophet import Prophet
m = Prophet()
m.fit(train)


## Listing 19-5. Creating a basic Prophet model

In [ ]:
future = m.make_future_dataframe(periods=len(test))
forecast = m.predict(future)


## Listing 19-6. Creating a basic Prophet model

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(list(test['y']), list(forecast.loc[450:,'yhat'] )))


## Listing 19-7. Plotting the fit of the model

In [ ]:
import matplotlib.pyplot as plt
plt.plot(list(test['y']))
plt.plot(list(forecast.loc[450:,'yhat'] ))
plt.show()


## Listing 19-8. Creating a Prophet forecast plot

In [ ]:
fig1 = m.plot(forecast)
plt.show()


## Listing 19-9. Creating a Prophet decomposition plot

In [ ]:
fig2 = m.plot_components(forecast)
plt.show()


## Listing 19-10. Add montly seasonality to the plot

In [ ]:
m2 = Prophet()
m2.add_seasonality(name='monthly', period=30.5, fourier_order=5)

m2.fit(train)

future2 = m2.make_future_dataframe(periods=len(test))
forecast2 = m2.predict(future)
print(r2_score(list(test['y']), list(forecast2.loc[450:,'yhat'] )))

fig2 = m2.plot_components(forecast2)
plt.show()


## Listing 19-11. Prepare holidays data

In [ ]:
holidays = pd.read_csv('date_info.csv.zip')
holidays = holidays[holidays['holiday_flg'] == 1]
holidays = holidays[['calendar_date', 'holiday_flg']]
holidays = holidays.drop(['holiday_flg'], axis=1)
holidays['holiday'] = 'holiday'
holidays.columns = ['ds', 'holiday']


## Listing 19-12. Add holidays to the model

In [ ]:
m3 = Prophet(holidays=holidays)
m3.fit(train)
future3 = m3.make_future_dataframe(periods=len(test))
forecast3 = m3.predict(future)

print(r2_score(list(test['y']), list(forecast3.loc[450:,'yhat'] )))

fig2 = m3.plot_components(forecast3)
plt.show()


## Listing 19-13. Add reservations to the model

In [ ]:
X_reservations = pd.read_csv('air_reserve.csv.zip')
X_reservations['visit_date'] = pd.to_datetime(X_reservations['visit_datetime']).dt.date
X_reservations = pd.DataFrame(X_reservations.groupby('visit_date')['reserve_visitors'].sum())
X_reservations = X_reservations.reset_index(drop = False)
train4 = train.copy()
train4['ds'] = pd.to_datetime(train4['ds']).dt.date
train4 = train4.merge(X_reservations, left_on = 'ds', right_on = 'visit_date', how = 'left')[['ds', 'y', 'reserve_visitors']].fillna(0)


## Listing 19-14. Add reservations to the model

In [ ]:
m4 = Prophet()
m4.add_regressor('reserve_visitors')
m4.fit(train4)
future4 = m4.make_future_dataframe(periods=len(test))
future4['ds'] = pd.to_datetime(future4['ds']).dt.date

future4 = future4.merge(X_reservations, left_on = 'ds', right_on = 'visit_date', how = 'left')[['ds', 'reserve_visitors']].fillna(0)

forecast4 = m4.predict(future4)

print(r2_score(list(test['y']), list(forecast4.loc[450:,'yhat'] )))

plt.plot(list(test['y']))
plt.plot(list(forecast4.loc[450:,'yhat'] ))

fig2 = m4.plot_components(forecast4)
plt.show()


## Listing 19-15. Grid Searching the Prophet

In [ ]:
def model_test(holidays, weekly_seasonality,
yearly_seasonality, add_monthly, add_reserve, changepoint_prior_scale, holidays_prior_scale, month_fourier):
    
    m4 = Prophet(
yearly_seasonality=yearly_seasonality, 
weekly_seasonality=weekly_seasonality, 
holidays=holidays, 
changepoint_prior_scale=changepoint_prior_scale, 
holidays_prior_scale=holidays_prior_scale)
    
    if add_monthly:    
        m4.add_seasonality(
name='monthly', 
period=30.5, 
fourier_order=month_fourier)
    
    if add_reserve:
        m4.add_regressor('reserve_visitors')

    m4.fit(train4)

    future4 = m4.make_future_dataframe(periods=len(test))

    future4['ds'] = pd.to_datetime(future4['ds']).dt.date
    
    if add_reserve:
        future4 = future4.merge(
X_reservations, 
left_on = 'ds', 
right_on = 'visit_date', 
how = 'left')
   future4 = future4[['ds', 'reserve_visitors']]
   future4 = future4.fillna(0)

    forecast4 = m4.predict(future4)

    return r2_score(
list(test['y']),
list(forecast4.loc[450:,'yhat'] )) 
# Setting the grid
holidays_opt = [holidays, None]
weekly_seas = [ 5, 10, 30, 50]
yearly_seas = [ 5, 10, 30, 50]
add_monthly = [True, False]
add_reserve = [True, False]
changepoint_prior_scale = [0.1, 0.3, 0.5]
holidays_prior_scale = [0.1, 0.3, 0.5]
month_fourier = [5, 10, 30, 50]

# Looping through the grid
grid_results = []
for h in holidays_opt:
  for w in weekly_seas:
    for ys in yearly_seas:
      for m in add_monthly:
        for r in add_reserve:
           for c in changepoint_prior_scale:
             for hp in holidays_prior_scale:
               for mf in month_fourier:
                  r2=model_test(h,w,ys,m,r,c,hp,mf)
                  print([w,ys,m,r,c,hp,mf,r2])
                  grid_results.append([h,w,ys,m,r,c,hp,mf,r2])

# adding it all to a dataframe and extract the best model
benchmark = pd.DataFrame(grid_results)
benchmark = benchmark.sort_values(8, ascending=False)

h, w,ys, m, r, c,hp,mf,r2 = list(benchmark.iloc[0,:])

# Fit the Prophet with those best hyperparameters
m4 = Prophet(
yearly_seasonality=ys, 
weekly_seasonality=w, 
holidays=h, 
changepoint_prior_scale=c, 
holidays_prior_scale=hp)
    
if m:    
    m4.add_seasonality(
name='monthly', 
period=30.5, 
fourier_order=mf)

if r:
    m4.add_regressor('reserve_visitors')

m4.fit(train4)

future4 = m4.make_future_dataframe(periods=len(test))

future4['ds'] = pd.to_datetime(future4['ds']).dt.date

if r:
    future4 = future4.merge(
X_reservations, 
left_on = 'ds', 
right_on = 'visit_date', 
how = 'left')
    future4 = future4[['ds', 'reserve_visitors']]
    future4 = future4.fillna(0)

forecast4 = m4.predict(future4)
